In [1]:
import pandas as pd
from utils import transformers_bert_completions, load_splits, load_models
from utils_model_sampling import sample_across_models, hyperparameter_utils

from yyy_analysis import examples_figure
import numpy as np

import os
from os.path import join, exists


import childespy
%pdb

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 20614 bytes (20 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

Automatic pdb calling has been turned ON


In [2]:
 os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
import configuration
config = configuration.Config()

### Get general data

In [4]:
# Make this config regenerate controlled later

pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"', db_version = "2020.1").iloc[0]['id']

regenerate = False
this_path = join(config.prov_csv_dir, 'pvd_utt_glosses.csv')

if regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, type from utterance where corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")
    utt_glosses.to_csv(this_path, index=False)
else: 
    utt_glosses = pd.read_csv(this_path)

R[write to console]: Using supported database version: '2020.1'.



### Load example information

In [5]:
all_tokens_phono = load_splits.load_phono()

In [6]:
success_idx = 16928243

all_tokens_phono.loc[all_tokens_phono.utterance_id == success_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]


,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
997717,I want to read,,,16928243,997717,310,42336
997718,I want to read,aə,aə,16928243,997718,310,42336
997719,I want to read,wɑn,wɑnt,16928243,997719,310,42336
997720,I want to read,də,tu,16928243,997720,310,42336
997721,I want to read,wid,ɹid,16928243,997721,310,42336
997722,I want to read,,,16928243,997722,310,42336


In [7]:
target_transcript_id = 42336 # Corresponds to the success_idx 

### Find new test examples - successes

In [8]:
# Changed this notebook to use the +/- 20 context newly generated versions.

# Written to match load_models.query_model_title
default_args = {
    'split' : 'all',
    'dataset' : 'all', 
    'is_tags' : False,
    'context_num' : 20,
}

childes_all_title = load_models.query_model_title(model_type = 'childes', **default_args)
adult_all_title = load_models.query_model_title(model_type = 'adult', **default_args)
unigram_title = 'CHILDES Unigram'

In [40]:
import imp
imp.reload(examples_figure)
imp.reload(transformers_bert_completions)
imp.reload(sample_across_models)

/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/sample_across_models.py:9: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()


<module 'utils_model_sampling.sample_across_models' from '/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/sample_across_models.py'>

In [56]:

# CDL + Context +/- 20 is needed
# BERT + Context +/- 20 is needed
# Childes on train data.

# How to load properly with sample across models?
which_models = [
    ('all', 'all', False, 0, 'data_unigram'),
    ('all', 'all', False, 20, 'childes'),
    ('all', 'all', False, 20, 'adult'),
]

raw_scores_across_models = examples_figure.get_scores_across_models(success_idx, which_models, True)
scores_across_models = pd.concat(raw_scores_across_models)

Running model CHILDES Unigram...
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Processing lambda value 1 of 20
In get_posteriors
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
In get_posteriors
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Running model CHILDES BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Processing lambda value 1 of 20
In get_posteriors
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
In get_posteriors


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running model Adult BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Processing lambda value 1 of 20
In get_posteriors
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
In get_posteriors


In [79]:
scores_across_models = scores_across_models.loc[scores_across_models.likelihood_type == 'wfst']

In [80]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type']]
success_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token,likelihood_type
997721,"CHILDES BERT without tags, , +-20 utts context",see go play look read know watch write mommy do,0.47244382544119096 0.09089431118000833 0.0311...,see go play look read know watch write mommy do,0.4724438 0.090894304 0.031178914 0.02277169 0...,0.018526,read,wfst


In [81]:
scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities','token','likelihood_type']]

,model,highest_posterior_words,highest_posterior_probabilities,token,likelihood_type
997721,"CHILDES BERT without tags, , +-20 utts context",see go play look read know watch write mommy do,0.47244382544119096 0.09089431118000833 0.0311...,read,wfst


In [82]:
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'see (0.47) go (0.09) play (0.03) look (0.02) read (0.02) know (0.02) watch (0.02) write (0.01) mommy (0.01) do (0.01)'

In [83]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'see (0.47) go (0.09) play (0.03) look (0.02) read (0.02) know (0.02) watch (0.02) write (0.01) mommy (0.01) do (0.01)'

In [84]:
success_example = scores_across_models.loc[(scores_across_models.model == adult_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'read (0.49) see (0.28) play (0.04) know (0.04) look (0.02) go (0.01) watch (0.01) help (0.01) try (0.01) talk (0.01)'

In [85]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'read (1.0) wait (0.0) weed (0.0) lead (0.0) watch (0.0) wonder (0.0) reading (0.0) reads (0.0) write (0.0) see (0.0)'

In [86]:
success_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token
997721,"Adult BERT without tags, , +-20 utts context",read wait weed lead watch wonder reading reads...,0.9995437315727504 0.00010855378542666844 5.85...,read see play know look go watch help try talk,0.4877993 0.27593085 0.036776084 0.036333278 0...,0.487799,read


In [87]:
success_example = scores_across_models.loc[(scores_across_models.model == unigram_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token
3,CHILDES Unigram,read with what would we weed one wait word wood,0.3947868604952227 0.24547081447729832 0.12217...,i a the yeah no it you and that this,0.04249687198055286 0.032549581544440515 0.027...,0.001047,read


In [88]:
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

In [89]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'read (0.39) with (0.25) what (0.12) would (0.04) we (0.03) weed (0.02) one (0.01) wait (0.01) word (0.01) wood (0.01)'

In [90]:
# Need to visualize utt_glosses -- how to do this? What is the dependency?
# Is this a providence one or?
utt_glosses.loc[(utt_glosses.transcript_id == target_transcript_id) &
                (utt_glosses.utterance_order.isin(range(310-10, 310+2)))] # Note to self: changed the range here

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
162628,Jasmine,42336,16928069,300,Lily,MOT,declarative
162639,a ballet,42336,16928081,301,Lily,CHI,declarative
162656,is Jasmine a ballerina,42336,16928098,302,Lily,MOT,question
162670,yeah,42336,16928113,303,Lily,CHI,declarative
162686,oh I didn't know that,42336,16928129,304,Lily,MOT,declarative
162710,I ready now yyy,42336,16928154,305,Lily,CHI,declarative
162730,whoa,42336,16928174,306,Lily,MOT,declarative
162744,yyy,42336,16928189,307,Lily,CHI,declarative
162760,yyy yyy,42336,16928205,308,Lily,CHI,declarative
162779,you want mamma let's see,42336,16928225,309,Lily,MOT,declarative


## Failure Example

In [91]:
yyy_idx = 16813515

raw_scores_across_models = examples_figure.get_scores_across_models(yyy_idx, which_models, False)
scores_across_models = pd.concat(raw_scores_across_models)

Running model CHILDES Unigram...
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Processing lambda value 1 of 20
In get_posteriors
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
In get_posteriors
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Running model CHILDES BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Processing lambda value 1 of 20
In get_posteriors
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
In get_posteriors


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running model Adult BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Processing lambda value 1 of 20
In get_posteriors
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
In get_posteriors


In [92]:
all_tokens_phono.loc[all_tokens_phono.utterance_id == yyy_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]

,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
289678,you make your yyy,,,16813515,289678,112,42253
289679,you make your yyy,ju,ju,16813515,289679,112,42253
289680,you make your yyy,meək,meək,16813515,289680,112,42253
289681,you make your yyy,jɜ,jɑɹ,16813515,289681,112,42253
289682,you make your yyy,fɜt,*,16813515,289682,112,42253
289683,you make your yyy,,,16813515,289683,112,42253


In [93]:
yyy_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type']]
yyy_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token,likelihood_type
289682,"CHILDES BERT without tags, , +-20 utts context",own house bed dinner mouth breakfast hand name...,0.20273877491755657 0.07377459785277564 0.0380...,own house bed dinner mouth breakfast hand name...,0.20273878 0.0737746 0.038079448 0.03176582 0....,NaN,NaN,wfst


In [94]:
words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))])

'own (0.2) house (0.07) bed (0.04) dinner (0.03) mouth (0.02) breakfast (0.02) hand (0.01) name (0.01) mess (0.01) tea (0.01)'

In [95]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'own (0.2) house (0.07) bed (0.04) dinner (0.03) mouth (0.02) breakfast (0.02) hand (0.01) name (0.01) mess (0.01) tea (0.01)'

In [96]:
yyy_example = scores_across_models.loc[(scores_across_models.model == adult_all_title ) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'own (0.25) choice (0.24) point (0.04) bed (0.03) call (0.03) guess (0.03) wish (0.02) choices (0.02) move (0.02) mistake (0.02)'

In [97]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'favorite (0.33) first (0.25) effort (0.13) fort (0.05) offer (0.04) threat (0.02) fist (0.02) fate (0.01) point (0.01) fit (0.01)'

In [98]:
yyy_example = scores_across_models.loc[(scores_across_models.model == unigram_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

In [99]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'for (0.4) fit (0.12) first (0.1) front (0.05) foot (0.05) different (0.03) feet (0.03) favorite (0.02) fruit (0.01) forty (0.01)'

In [100]:
# Get the surrounding context
utt_glosses.loc[(utt_glosses.transcript_id == 42253) &
                (utt_glosses.utterance_order.isin(range(112-3, 112+3)))]

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
50658,then we won't be able to put them back into th...,42253,16813459,109,Alex,MOT,declarative
50679,do you want ta put some beans in your eggs and...,42253,16813482,110,Alex,MOT,question
50697,no,42253,16813501,111,Alex,CHI,declarative
50710,you make your yyy,42253,16813515,112,Alex,CHI,declarative
50723,can I make one,42253,16813529,113,Alex,MOT,question
50745,no,42253,16813554,114,Alex,MOT,declarative
